In [1]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
from qiskit.optimization.applications.ising import docplex
# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np

from collections import OrderedDict

In [2]:
M = [2,6,3]
K = [1,4,2]
T = [12, 6, 24]
d = 19

def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m

time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print(time_matrix)
print(cost_matrix)

n1  = np.shape(cost_matrix)[0]
n2 = np.shape(cost_matrix)[1]

[[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
[[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [3]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        print(binary_string)
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0

optimal_key = "0000001000"
time_limit = 19

def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0
     
    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1
            
    print('most likely solution: ',sample_most_likely(dict_res))      
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)

def is_correct(key):
    reversed_key = key[::-1]
    if solution_vector_correct(reversed_key) and execution_time(reversed_key) <= time_limit:
        return True
    return False

correct_machines = ['00', '01', '11']
machine_to_index = {'00': 0, '01': 1, '11': 2}

def execution_time(k):
    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1
    
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])
    
    # for vector "10" (the only incorrect one), assign sum of all machines times for task
    task_time_sum =  np.sum(time_matrix, axis=0) 
    task1_time = task_time_sum[0] if task1_machine == None else time_matrix[task1_machine, 0]
    task2_time = task_time_sum[1] if task2_machine == None else time_matrix[task2_machine, 1]
    task3_time = task_time_sum[2] if task3_machine == None else time_matrix[task3_machine, 2]
    
    return slack_sum + task1_time + task2_time + task3_time
    
def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    if task1_machine in correct_machines and task2_machine in correct_machines and task3_machine in correct_machines:
        return True
    return False

In [4]:
#Minimalizacja kosztu
def getMdl(A,B,C):
    mdl = Model(name='workflow1')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    
    objective = A* mdl.sum(cost_matrix[2,i] * x[(i,0)]   
                            + cost_matrix[1,i] * (x[(i,1)] - x[(i,0)]) ** 2
                            + cost_matrix[0,i] * (1 - x[(i,1)]) for i in range(0, 3))
    
    mdl.add_constraint(B * mdl.sum((x[(i,0)] - x[(i,0)] * x[(i,1)]) for i in range(0, 3)) == 0)

    mdl.add_constraint(C * (mdl.sum([time_matrix[2,i] * x[(i,0)]   
                            + time_matrix[1,i] * (x[(i,1)] - x[(i,0)])
                            + time_matrix[0,i] * (1 - x[(i,1)]) for i in range(0, 3)]) 
                                  + (8 * x[7] + 4 * x[8] + 2 * x[9] + x[10])) == C * d)
    
    mdl.minimize(objective)
    H, _ = docplex.get_operator(mdl)
    return H

In [5]:
write_to_file = False
filename = 'results.txt'

seed = 123569
aqua_globals.random_seed = seed
entanglement='full'

reps = 2
max_trials = 1000
shots = 100

A = 2
B = 10
C = 7

qubitOp = getMdl(A, B, C)
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubitOp.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubitOp, ry, spsa)
# provider= IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = vqe.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

[('0110001011', 1), ('1100000100', 1), ('1100000110', 1), ('1100001100', 1), ('1100101100', 1), ('0001100011', 1), ('0001100100', 1), ('0001100101', 1), ('0001100110', 1), ('0001101010', 1), ('0001101011', 1), ('0000001011', 1), ('0000001110', 1), ('1100111011', 2), ('0001100001', 2), ('0001101001', 2), ('0101100000', 3), ('0101100111', 3), ('1000101011', 3), ('1100001010', 4), ('1100101011', 4), ('0001100010', 7), ('1100001011', 9), ('0001100111', 11), ('0001101000', 18), ('0001100000', 19)]
most likely solution:  (array([0, 0, 0, 0, 0, 1, 1, 0, 0, 0]), 19)
optimal:  0
correct solutions:  44
incorrect solutions:  56
correct configs:  12
incorrect configs:  14


In [6]:
# vqe.get_optimal_circuit().draw()

In [7]:
vqe.get_optimal_cost()

-1251019.2399999998

In [8]:
vqe.get_optimal_vector()

{'0101100000': 3,
 '0101100111': 3,
 '0110001011': 1,
 '1000101011': 3,
 '1100000100': 1,
 '1100000110': 1,
 '1100001010': 4,
 '1100001011': 9,
 '1100001100': 1,
 '1100101011': 4,
 '1100101100': 1,
 '1100111011': 2,
 '0001100000': 19,
 '0001100001': 2,
 '0001100010': 7,
 '0001100011': 1,
 '0001100100': 1,
 '0001100101': 1,
 '0001100110': 1,
 '0001100111': 11,
 '0001101000': 18,
 '0001101001': 2,
 '0001101010': 1,
 '0001101011': 1,
 '0000001011': 1,
 '0000001110': 1}